In [ ]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    !git clone https://github.com/unionai-oss/bert-llm-classification-pipeline
    %cd bert-llm-classification-pipeline
    !pip install -r requirements.txt

### 🔐 Authenticate
To use **Union.ai**, you'll need to authenticate your account. Follow the appropriate step based on your setup:  

##### 🔸 **Using Union BYOC Enterprise**  

If you're using a **[Union BYOC Enterprise](https://www.union.ai/pricing)** account, log in with the following command:  
```bash
union create login --host <union-host-url>
```

Replace <union-host-url> with your organization's Union instance URL.

##### 🔸 Using Union Serverless
If you're using [Union Serverless](https://www.union.ai/) , authenticate by running the command below:

Create an account for free at [Union.ai](https://union.ai) if you don't have one yet:
 

In [ ]:
# 🌟 Authenticate to union serverless
!union create login --serverless --auth device-flow

## Training Faster RCNN Object Detetection Model Pipeline

In [ ]:
!union run --remote workflows/train-frcnn-pipeline.py faster_rcnn_train_workflow --epochs 3

In [ ]:
%%writefile workflow.py



In [ ]:
# containers.py

In [ ]:
# requirmets.txt

In [ ]:
# data tasks


In [ ]:
# model tasks

## Run Model Locally

 lets pull donw our same datset locally to run the model on examples

In [ ]:
# hub download data

def download_hf_dataset(repo_id: str = 'sagecodes/union_swag_coco',
                        local_dir: str = "dataset",
                        sub_folder: str = "swag"):
    
    from huggingface_hub import snapshot_download

    if local_dir:
        dataset_dir = os.path.join(local_dir)
        os.makedirs(dataset_dir, exist_ok=True)

    # Download the dataset repository
    repo_path = snapshot_download(repo_id=repo_id, 
                                  repo_type="dataset",
                                  local_dir=local_dir)
    if sub_folder:
        repo_path = os.path.join(repo_path, sub_folder)
        # use sub_folder to return a specific folder from the dataset

    print(f"Dataset downloaded to {repo_path}")

    print(f"Files in dataset directory: {os.listdir(repo_path)}")

    return repo_path

download_hf_dataset()

In [ ]:
# Model download

In [ ]:
# Draw boxes on images

# Define labels map
labels_map = {1: "Union Sticker", 2: "Flyte Sticker"}

# Check and set the available device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


font_url = 'https://github.com/google/fonts/raw/main/apache/robotomono/RobotoMono%5Bwght%5D.ttf'
response = requests.get(font_url)
font = ImageFont.truetype(BytesIO(response.content), size=20)


# Function to draw bounding boxes
def draw_boxes(image, boxes, labels, scores, labels_map):
    draw = ImageDraw.Draw(image, 'RGBA')
    # font = ImageFont.truetype(urlopen(truetype_url), size=20)
    # font = ImageFont.load_default() # default font in pil


    colors = {
        0: (255, 173, 10, 200),  # Class 0 color (e.g., blue)
        1: (28, 140, 252, 200),  # Class 1 color (e.g., orange)
    }
    colors_fill = {
        0: (255, 173, 10, 100),  # Class 0 fill color (e.g., bluea)
        1: (28, 140, 252, 100),  # Class 1 fill color (e.g., orangea)
    }

    for box, label, score in zip(boxes, labels, scores):
        if score > 0.6: # adjust threshold as needed
          color = colors.get(label, (0, 255, 0, 200))
          fill_color = colors_fill.get(label, (0, 255, 0, 100))
          draw.rectangle([(box[0], box[1]), (box[2], box[3])], outline=color, width=3)
          draw.rectangle([(box[0], box[1]), (box[2], box[3])], fill=fill_color)
          label_text = f"{labels_map[label]}: {score:.2f}"
          text_size = font.getbbox(label_text)
          draw.rectangle([(box[0], box[1] - text_size[1]), (box[0] + text_size[0], box[1])], fill=color)
          draw.text((box[0], box[1] - text_size[1]), label_text, fill="white", font=font)

    return image


# Load a single test image
image_path = '/content/dataset/swag/images/1bd5a6b5-20240916_133544.jpg'
image = Image.open(image_path).convert("RGB")
image_tensor = F.to_tensor(image).unsqueeze(0).to(device)

# Run inference
with torch.no_grad():
    outputs = model(image_tensor)

# Get the boxes, labels, and scores
boxes = outputs[0]['boxes'].cpu().numpy()
labels = outputs[0]['labels'].cpu().numpy()
scores = outputs[0]['scores'].cpu().numpy()

# Define labels map
labels_map = {0: "Background", 1: "Union Sticker", 2: "Flyte Sticker"}

# Draw the boxes on the image
image_with_boxes = draw_boxes(image, boxes, labels, scores, labels_map)

# Display the image
image_with_boxes.show()

# Save the image
image_with_boxes.save('output_image.jpg')

In [ ]:
# run inference on image

In [ ]:
# run inference on video frames

In [ ]:
# Webcam example

## Serving as an App on Union

We can serve our model as an app on Union. This allows us to run the model in a production environment and make it available for use by other applications or users.

This example will use Gradio, but we could also use any other web framework like Flask or FastAPI to serve our model as an API. 